In [ ]:
# Importing Utilities
import openai
from langchain_openai import AzureOpenAIEmbeddings
import os
from geopy.geocoders import Nominatim
import requests

In [ ]:
# Setting OpenAI Credential
openai.api_version = "2024-02-01"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_type = 'azure'
os.environ["OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")

In [ ]:
# OpenAI Initialization
azure_openai_client = openai.AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"), 
    # api_version = get_secret('GprntAICTDevOpenAIVersion'), 
    api_version = "2024-02-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)


In [ ]:
# Prompt Engineering
prompt = '''
Answer the following question.
     you are traveling and outing guider bot.
     Generate answer with bullet points with reasoning of it's good.

      ---
      QUESTION: 
      {query}
      ---

      Current Location and weather:
      {user_location_weather}

      ---
      Conversation History:
      {conversation_history}
      ---
'''




# Prompt Engineering
prompt_suggestion = '''
give me follow up suggestive questions in python list according to last query, response and previous conversation and current location and weather.
<Query>:{question}
<response>:{response}
<previous conversation>: 
{conversation_history}
Current Location and weather:
{user_location_weather}

'''


In [ ]:

def suggestion(user_input,response,conversation_history, user_location_weather):

    # complete_search = pair[0] + pair[1]
    # docs = vector_store_obj.similarity_search(query=complete_search, k=8, search_type='similarity')
    
    # Send prompt to LLM
    data = azure_openai_client.chat.completions.create(
        model="gpt-4", 
        max_tokens=300, 
        temperature=0.0,
        messages=[
            {'role': 'system', 'content': 'You\'re a Greenprint QnA Assistant'}, 
            {'role': 'user', 'content': prompt_suggestion.format(question=user_input, response=response, conversation_history=conversation_history, user_location_weather=user_location_weather)}
        ], 
        stream=False
    )
    
    return data.choices[0].message.content




In [ ]:
def location_and_weather(latitude, longitude):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    address = location.raw['address']
    
    city = address.get('city', '')
    state = address.get('state', '')
    country = address.get('country', '')
    postcode = address.get('postcode', '')

    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}¤t_weather=true"
    response = requests.get(url)
    data = response.json()
    
    if 'current_weather' in data:
        current_weather = data['current_weather']
        temp = current_weather['temperature']
        windspeed = current_weather['windspeed']
        winddirection = current_weather['winddirection']
        weathercode = current_weather['weathercode']
        
        return {
            "city": city,
            "state": state,
            "country": country,
            "postcode": postcode,
            "temperature": temp,
            "windspeed": windspeed,
            "winddirection": winddirection,
            "weathercode": weathercode
        }
    else:
        return {"error": data.get("reason", "Failed to fetch weather data")}


In [ ]:

def bot():
    
    # Initialize conversation history
    conversation_history = []
    # reference_time = datetime.datetime.now()

    while True:
            
            # current_time = datetime.datetime.now()
            # time_difference = current_time - reference_time
            # time_difference_minutes = time_difference.total_seconds() / 60  # Convert seconds to minutes
            # if time_difference_minutes < 1:
                    # Get user input
                user_input = input("You: ")

                if user_input=="e" or user_input=="stop":
                    break

                print("Question: ",user_input)
                # print("\n")
                # docs = vector_store_obj.similarity_search(query=user_input, k=8, search_type='similarity')

                # Calculate total token length of prompt
                prompt_token_length = sum(len(chunk["content"].split()) for chunk in conversation_history)
                prompt_token_length += len(user_input) + 300
                # print(conversation_history)
                # If prompt token length exceeds 8000, remove conversations from the beginning
                while prompt_token_length > 8000:
                    removed_chunk = conversation_history.pop(0)
                    prompt_token_length -= len(removed_chunk["content"].split())

                # Send prompt to LLM
                data = azure_openai_client.chat.completions.create(
                    model="gpt-4", 
                    max_tokens=300, 
                    temperature=0.0,
                    messages=[
                        {'role': 'system', 'content': 'You\'re a QnA Assistant'}, 
                        {'role': 'user', 'content': prompt.format(query=user_input, conversation_history=conversation_history, user_location_weather=user_location_weather)}
                    ], 
                    stream=True
                )
                # print(data.choices[0].message.content)
                s=""
                print("Answer: ")
                # Streaming Print
                for chunk in data:
                    try:
                        if chunk.choices[-1].delta.content != None:
                            print(chunk.choices[-1].delta.content, end='')
                            if not None:
                                s += chunk.choices[-1].delta.content
                    except Exception as e:
                        pass

                
                # Update conversation history
                conversation_history.append({"role": "user", "content": user_input})
                conversation_history.append({"role": "system", "content": s})
                suggestions=suggestion(user_input,s,conversation_history, user_location_weather)
                print("\n")
                print("Suggestion : ")
                print(suggestions)

                print("\n")
                # Define a reference time
                # reference_time = datetime.datetime.now()

                    
                # Limit conversation history size (optional)
                # conversation_history = conversation_history[-10:] 

            # else:
            #     flag=input("would you want to continue(yes/no): ")
            #     if flag=="yes":
            #         current_time = datetime.datetime.now()

            #     else:
            #         break






In [ ]:
bot()